In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
image_path = r'C:\Users\giofr\Desktop\NUMERICAL ANALYSIS FOR MACHINE LEARNING\TDE\7 FEBRUARY 2023\polimi_compress.jpg'
img = imread(image_path)
img_RGB = [np.array(img[:,:,i], dtype = np.float64)/255 for i in range(3)]

def plot_image(RGB, ax = None):
  if ax is None:
    _, ax = plt.subplots(1,1, figsize = (12,5))
  ax.imshow(np.clip(np.stack(RGB, axis = 2),0,1))
  plt.axis('off')

plot_image(img_RGB)

# Now, randomly replace 70% of the pixels with random colors.
damage_fraction = 0.7
np.random.seed(0)

mask_remove = np.random.choice(a=[True, False], size=(img.shape[0],img.shape[1]), p=[damage_fraction, 1-damage_fraction])
mask_keep = np.logical_not(mask_remove)
img_damaged_RGB = [img_RGB[i].copy() for i in range(3)]
for i in range(3):
  img_damaged_RGB[i][mask_remove] = np.random.rand(np.sum(mask_remove))

In [ ]:
# display the damaged imagine
plot_image(img_damaged_RGB)

In [ ]:
# implement SVT algorithm to reconstruct the original matrix
def SVT(matrix_channel, threshold, mask_keep):
  n_max_iter = 200
  tol = 1e-6
  A = matrix_channel.copy()

  for i in range(n_max_iter):
    A_old = A.copy()
    U,s,VT = np.linalg.svd(A, full_matrices = False)

    s[s < threshold] = 0
    A = U @ np.diag(s) @ VT
    A[mask_keep] = matrix_channel[mask_keep]
    increment = np.linalg.norm(A - A_old)

    if increment < tol:
        print("Tolerance achieved!")
        break

    print("n_iter: %d" % i)
    return A


In [ ]:
thresholds = list(range(2,2,22))
reconstructed_imgs = list()

for threshold in thresholds:
  img_reconstructed = img_damaged_RGB.copy()
  print("current threshold: %d" %threshold)
  for i in range(3):
    img_reconstructed[i] = SVT(img_damaged_RGB[i], threshold, mask_keep)

  reconstructed_imgs.append(img_reconstructed)


In [ ]:
# finally print the resulting image against the original one
fig, axes = plt.subplots(nrows=len(thresholds), ncols=3, figsize = (18,6))
for i in range(len(thresholds)):
  plot_image(reconstructed_imgs[i], axes[3*i])
  axes[3*i].set_title(thresholds[i])
  plot_image(img_damaged_RGB, axes[3*i + 1])
  plot_image(img_RGB, axes[3*i + 2])